In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode,iplot

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df=pd.read_csv("/kaggle/input/ipl-complete-dataset-20082020/IPL Matches 2008-2020.csv")
df_balltoball=pd.read_csv("/kaggle/input/ipl-complete-dataset-20082020/IPL Ball-by-Ball 2008-2020.csv")

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
x=['Sunrisers Hyderabad', 'Mumbai Indians', 'Gujarat Lions',
    'Rising Pune Supergiant', 'Royal Challengers Bangalore',
    'Kolkata Knight Riders', 'Delhi Daredevils', 'Kings XI Punjab',
    'Chennai Super Kings', 'Rajasthan Royals', 'Deccan Chargers',
    'Kochi Tuskers Kerala', 'Pune Warriors', 'Rising Pune Supergiants', 'Delhi Capitals']

y = ['SRH','MI','GL','RPS','RCB','KKR','DC','KXIP','CSK','RR','SRH','KTK','PW','RPS','DC']

df.replace(x,y,inplace = True)
df_balltoball.replace(x,y,inplace = True)

In [ ]:
df.head()

In [ ]:
def Annotation(plots):
    for bar in plots.patches:

        plots.annotate(format(bar.get_height(), '.2f'), 
                       (bar.get_x() + bar.get_width() / 2, 
                        bar.get_height()), ha='center', va='center',
                       size=8, xytext=(0, 5),
                       textcoords='offset points')

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20,8), sharey=True)
sns.set_theme(style="whitegrid")

sns.countplot(ax=ax[0],x="winner", data=df,order = df["winner"].value_counts().index)
Annotation(sns.countplot(ax=ax[0],x="winner", data=df,order = df["winner"].value_counts().index))
ax[0].set_title("Match Winner")


sns.countplot(ax=ax[1],x="toss_winner", data=df,order = df["toss_winner"].value_counts().index)
Annotation(sns.countplot(ax=ax[1],x="toss_winner", data=df,order = df["toss_winner"].value_counts().index))
ax[1].set_title("Toss Winner")

sns.countplot(ax=ax[2],x="winner", data=df[df['toss_winner']==df['winner']],order = df["winner"].value_counts().index)
Annotation(sns.countplot(ax=ax[2],x="winner", data=df[df['toss_winner']==df['winner']],order = df["winner"].value_counts().index))
ax[2].set_title("Toss winner and Match Winner")

In [ ]:
df1 = df.groupby('toss_winner')['toss_decision'].value_counts(normalize=True)
df1 = df1.mul(100)
df1 = df1.rename('percent').reset_index()


g = sns.catplot(x='toss_winner',y='percent',hue='toss_decision',kind='bar',data=df1)
g.fig.set_figwidth(20)
g.fig.set_figheight(8)
g.ax.set_ylim(0,100)


for p in g.ax.patches:
    txt = str(p.get_height().round()) + '%'
    txt_x = p.get_x() 
    txt_y = p.get_height()
    g.ax.text(txt_x,txt_y,txt)


In [ ]:
teams=pd.concat([df['team1'],df['team2']])
teams=teams.value_counts().reset_index()
teams.columns=['Team','Total Matches']
teams['Wins']=df['winner'].value_counts().reset_index()['winner']
teams['Win percentage(%)']=((teams['Wins']/teams['Total Matches'])*100).round()
trace = go.Table(
    header=dict(values=["Team","Total Matches","Wins",'Win percentage(%)'],
                fill = dict(color='#CD5C5C'),
                font = dict(color=['rgb(45, 45, 45)'] * 5, size=14),
                align = ['center'],
               height = 30),
    cells=dict(values=[teams['Team'], teams['Total Matches'], teams['Wins'],teams['Win percentage(%)']],
               fill = dict(color=['rgb(174, 214, 241)', 'rgba(228, 222, 249, 0.65)']),
               align = ['center'], font_size=13, height=25))

layout = dict(
    width=850,
    height=520,
    autosize=False,
    title='Total Matches vs Wins per team',
    margin = dict(t=100),
    showlegend=False,    
)
fig1 = dict(data=[trace], layout=layout)
iplot(fig1)